In [1]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [230]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same',input_shape=[15,30,1]),
    layers.MaxPool2D(),
    layers.Dropout(.3),
    #Block 2
    layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),layers.Dropout(.3),

    #Third hidden layer
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same'),
    #layers.LSTM(25,input_shape=[13,13,1]),
    #output 
    layers.Flatten(),
    layers.Dense(units=169,activation="relu"),
    layers.Dense(units=3,activation="sigmoid"),
    #layers.Reshape([1152,1]),
    #layers.Dense(units=169,activation="relu"),
    #layers.Dense(units=169,activation="sigmoid")
])
#model.add(keras.layers.TimeDistributed(model))
#model.add(layers.LSTM(16))

#model.add(layers.Dense(units=48,activation="relu"))
#model.add(layers.Dense(units=16,activation="softmax"))
#model.add(layers.Reshape([16,3]))
#model.add(layers.Dense(units=169,activation="sigmoid"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 15, 30, 128)       1280      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 15, 128)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 7, 15, 128)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 15, 128)        147584    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 3, 7, 128)        

1+1

In [4]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
targets=df['path']
startshape=df['shape_mapping']





In [52]:
import random as random
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
seq=df['sequence']
length=16;
trainx=[]
for i in range(len(seq)):


    seq[i]=seq[i].replace("P","1")
    seq[i]=seq[i].replace("H","2")
    seq[i]=seq[i].ljust(length,'0')
onehotencode=np.zeros([len(seq),3])
for i in range(len(seq)):

        onehotencode[i][int(seq[i][9])]=1
starts=df['path']   
arraysize=13
for i in range(len(seq)):
    c=shape_helper.deserialize_path(starts[i])
    d=path_to_shape(c,arraysize)
    trainx.append(d[0])
#seq = [f"{x:0{length}}" for x in seq]
validshapes=[]
validseqs=[]
for i in range(len(trainx)):
    prob=random.random()
    eve=len(trainx)
    if prob<.05:

        validshapes.append(trainx[eve-i])
        validseqs.append(onehotencode[eve-i])
        trainx=np.delete(trainx,eve-i,0)
        onehotencode=np.delete(onehotencode,eve-i,0)

    


/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_42796/2460639614.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("P","1")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_42796/2460639614.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("H","2")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_42796/2460639614.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [319]:
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
seq=df['sequence']
length=16;
trainx=[]
for i in range(len(seq)):


    seq[i]=seq[i].replace("P","1")
    seq[i]=seq[i].replace("H","2")
    seq[i]=seq[i].ljust(length,'0')
onehotencode=np.zeros([len(seq),length,3])
for i in range(len(seq)):
    for j in range(len(seq[1])):

        onehotencode[i][j][int(seq[i][j])]=1
starts=df['path']   
arraysize=13
for i in range(len(seq)):
    c=shape_helper.deserialize_path(starts[i])
    d=path_to_shape(c,arraysize)
    trainx.append(d[0])
#seq = [f"{x:0{length}}" for x in seq]
validshapes=[]
validseqs=[]
for i in range(len(trainx)):
    prob=random.random()
    eve=len(trainx)
    if prob<.05:

        validshapes.append(trainx[eve-i])
        validseqs.append(onehotencode[eve-i])
        trainx=np.delete(trainx,eve-i,0)
        onehotencode=np.delete(onehotencode,eve-i,0)
expandedx=[]
expandedy=[]
nextouty=[]
for j in range(len(onehotencode)):
    po=np.zeros([16,16,3])
    for x in range(len(onehotencode[j])):
        expandedx.append(trainx[j])
        for k in range(x):
            po[x][k]=onehotencode[j][k]
        nextouty.append(onehotencode[j][x])
        expandedy.append(po[x])


expandvalx=[]
expandvaly=[]
for i in range(len(trainx)):
    prob=random.random()
    eve=len(trainx)
    if prob<.05:

        validshapes.append(trainx[eve-i])
        validseqs.append(onehotencode[eve-i])
        trainx=np.delete(trainx,eve-i,0)
        onehotencode=np.delete(onehotencode,eve-i,0)


/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_44865/1824741830.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("P","1")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_44865/1824741830.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("H","2")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_44865/1824741830.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [320]:
#turn your expanded y into another array to superimpose on the path matrix
matexy=[]
for i in range(len(expandedy)):
    zen=np.zeros([len(expandedx[i]),len(expandedx[i])])
    for j in range(len(expandedy[i])):
        if (np.where(expandedy[i][j]==1)!=0):
            if(expandedy[i][j][1]==1 or expandedy[i][j][2]==1):
                if (len(np.where(expandedx[i]==j))!=0):
                    x,y=np.where(expandedx[i]==j+1)
                    if len(x)!=0:
                        x=x[0]
                        y=y[0]
                        
 
                        zen[x][y]=np.where(expandedy[i][j]==1)[0][0]
    matexy.append(zen)

In [321]:
xtrain=tf.concat([matexy,expandedx],-1)
nextouty=tf.convert_to_tensor(nextouty)


In [322]:
#so given validshapes and validseqs, turn these into the same thing as nextouty and xtrain then take a sample 
expandvalx=[]
expandvaly=[]
nextvaly=[]
for j in range(len(validseqs)):
    po=np.zeros([16,16,3])
    for x in range(len(validseqs[j])):
        expandvalx.append(validshapes[j])
        for k in range(x):
            po[x][k]=validseqs[j][k]
        nextvaly.append(validseqs[j][x])
        expandvaly.append(po[x])
mateval=[]
for i in range(len(expandvaly)):
    zen=np.zeros([len(expandvalx[i]),len(expandvalx[i])])
    for j in range(len(expandvaly[i])):
        if (np.where(expandvaly[i][j]==1)!=0):
            if(expandvaly[i][j][1]==1 or expandvaly[i][j][2]==1):
                if (len(np.where(expandvalx[i]==j))!=0):
                    x,y=np.where(expandvalx[i]==j+1)
                    if len(x)!=0:
                        x=x[0]
                        y=y[0]
                        
 
                        zen[x][y]=np.where(expandvaly[i][j]==1)[0][0]
    mateval.append(zen)
xval=tf.concat([mateval,expandvalx],-1)
nextvaly=tf.convert_to_tensor(nextvaly)


In [263]:
matexy[15]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 2., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 2., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [254]:
nextvaly

[array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([0., 0., 1.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([0., 0., 1.]),
 array([1., 0., 0.]),
 array([1., 0., 0.]),
 array([1.

In [5]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for i in range(len(path)):
        temp_grid[path[i][1]+round(grid_sizes/2)][path[i][0]+ round(grid_sizes/2)] = i+1
       # if coord==path[len(path)-1]:
            #temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = i
            # path transferred onto grid but uncentered
    #temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  

    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m+1,dev_n+1))
    x,y=np.where(temp_grid==2)
    start= (x+dev_m,y+dev_n)
    x,y=np.where(temp_grid==3)
    end=(x+dev_m,y+dev_n)
    
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = i
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start,end
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [ ]:
onehotencode

In [264]:
from tensorflow import keras
from tensorflow.keras import layers
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=.01),
    loss='CategoricalCrossentropy',metrics=['AUC']
)


In [362]:
history = model.fit(
    xtrain,nextouty,epochs=10
)


Epoch 1/10
2967/2967 [==============================] - 46s 15ms/step - loss: 0.1561 - auc: 0.9820
Epoch 2/10
2967/2967 [==============================] - 43s 15ms/step - loss: 0.1556 - auc: 0.9823
Epoch 3/10
2967/2967 [==============================] - 43s 14ms/step - loss: 0.1550 - auc: 0.9819
Epoch 4/10
2967/2967 [==============================] - 43s 14ms/step - loss: 0.1545 - auc: 0.9816
Epoch 5/10
2967/2967 [==============================] - 45s 15ms/step - loss: 0.1550 - auc: 0.9813
Epoch 6/10
2967/2967 [==============================] - 45s 15ms/step - loss: 0.1543 - auc: 0.9820
Epoch 7/10
2967/2967 [==============================] - 44s 15ms/step - loss: 0.1525 - auc: 0.9828
Epoch 8/10
2967/2967 [==============================] - 44s 15ms/step - loss: 0.1523 - auc: 0.9825
Epoch 9/10
2967/2967 [==============================] - 44s 15ms/step - loss: 0.1540 - auc: 0.9826
Epoch 10/10
2967/2967 [==============================] - 44s 15ms/step - loss: 0.1524 - auc: 0.9828


In [441]:
c=model.predict(xval)
threshold(c)
acc(c,nextvaly)/len(nextvaly)

322/322 [==============================] - 1s 5ms/step


0.14793934681181958

In [380]:
xval[0]

<tf.Tensor: shape=(15, 30), dtype=float64, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [437]:
test=tf.convert_to_tensor([xval[200]])
vals=np.zeros([16,3])


In [438]:
#print(vals)
for i in range(17): 
    x=model.predict(test)
    stepval=threshold(x)
    vals,o,test=update(expandvalx[200],stepval,vals)    
    test=tf.convert_to_tensor([test])



1/1 [==============================] - 0s 13ms/step
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [443]:
print(acc(vals,validseqs[200]))
#print(vals)
print(validseqs[200])
vals

22
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [303]:
#update function from past model.predict
np.where(xval[0]==1)
maybe=np.delete(xval[0],15,axis=1)

In [326]:
validseqs

[array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
      

In [ ]:
mateval=[]
for i in range(len(expandvaly)):
    zen=np.zeros([len(expandvalx[i]),len(expandvalx[i])])
    for j in range(len(expandvaly[i])):
        if (np.where(expandvaly[i][j]==1)!=0):
            if(expandvaly[i][j][1]==1 or expandvaly[i][j][2]==1):
                if (len(np.where(expandvalx[i]==j))!=0):
                    x,y=np.where(expandvalx[i]==j+1)
                    if len(x)!=0:
                        x=x[0]
                        y=y[0]
                        
 
                        zen[x][y]=np.where(expandvaly[i][j]==1)[0][0]
    mateval.append(zen)
xval=tf.concat([mateval,expandvalx],-1)

In [318]:
def update(tensor,newtuple,vals):
    zen=np.zeros([len(tensor),len(tensor)])
    for i in range(len(vals)):
        if (vals[i][0]+vals[i][1]+vals[i][2]==0):
            vals[i]=newtuple
            break
    for j in range(len(vals)):
        if (np.where(vals[j]==1)!=0):
            if(vals[j][1]==1 or vals[j][2]==1):
                if (len(np.where(tensor==j))!=0):
                    x,y=np.where(tensor==j+1)
                    if len(x)!=0:
                        x=x[0]
                        y=y[0]
                        zen[x][y]=np.where(vals[j]==1)[0][0]
    newentry=tf.concat([zen,tensor],-1)


    return vals,zen,newentry

In [309]:
expandvaly

[array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
      

In [316]:
update(expandvalx[0],threshold(x),expandvaly[0])

(array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 2., 2., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 2., 2., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [317]:
expandvalx[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 7, 6, 5, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 8, 1, 4, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [274]:
def acc(array1,array2):
    total=0
    for i in range(len(array1)):
        for j in range(len(array1[1])):
            if array1[i][j]!=array2[i][j]:
                total=total+1
    return total
def threshold(array1):
    for i in range(len(array1)):
        array1[i][np.where(array1[i]!=max(array1[i]))]=0
        array1[i][np.where(array1[i]==max(array1[i]))]=1
        array1[i][np.where(array1[i]!=max(array1[i]))]=0


    return array1
def accen(array1,array2):
    total=0
    for i in range(len(array1)):
        for j in range(len(array1[1])):
                if array1[i][j]!=array2[i][j]:
            
                    total=1+total
    return total
def thresholden(array1):
    for i in range(len(array1)):
        for j in range(len(array1[1])):
            array1[i][j][np.where(array1[i][j]==max(array1[i][j]))]=1

            array1[i][j][np.where(array1[i][j]!=max(array1[i][j]))]=0
        
    return array1



In [ ]:
trues=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]=trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
zs=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]=zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1


In [ ]:
#Our final accuracy statement
correct/len(valy)


In [ ]:
print(trues)
print(zs)

In [ ]:
cor=0
for i in range(len(thresx)):
    if(np.where(valy[i]==1)==np.where(thresx[i]==1)):
        print(i)
        cor=cor+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
cor

In [ ]:
for i in range(len(thresx)):
    
    print(np.where(thresx[i]==1))

In [ ]:
np.where(thresx[5]==1)[0]